In [2]:
import yfinance as yf
import pandas as pd
import numpy as np

In [3]:
def fetch_stock_data(ticker: str = "NVDA", period: str = "60d", interval: str = "1d") -> pd.DataFrame:
    """
    Fetch historical stock data using yfinance.
    """
    stock = yf.Ticker(ticker)
    hist = stock.history(period=period, interval=interval)
    hist.reset_index(inplace=True)
    return hist

In [4]:
def compute_technical_indicators(df: pd.DataFrame) -> pd.DataFrame:
    """
    Compute RSI and MACD indicators and append to the stock data.
    """
    # RSI (14-day)
    delta = df['Close'].diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)
    avg_gain = pd.Series(gain).rolling(window=14).mean()
    avg_loss = pd.Series(loss).rolling(window=14).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    df['RSI_14'] = rsi

    # MACD (12-day EMA - 26-day EMA)
    ema_12 = df['Close'].ewm(span=12, adjust=False).mean()
    ema_26 = df['Close'].ewm(span=26, adjust=False).mean()
    macd = ema_12 - ema_26
    signal = macd.ewm(span=9, adjust=False).mean()
    df['MACD'] = macd
    df['MACD_Signal'] = signal
    df['MACD_Hist'] = df['MACD'] - df['MACD_Signal']

    return df

In [5]:
def nvda_data_pipeline():
    """
    Full pipeline to get NVDA stock data and compute technical indicators.
    """
    df = fetch_stock_data("NVDA", "60d", "1d")
    enriched_df = compute_technical_indicators(df)
    return enriched_df

# Run it
if __name__ == "__main__":
    df = nvda_data_pipeline()
    print(df.tail())

                        Date        Open        High         Low       Close  \
55 2025-07-30 00:00:00-04:00  176.509995  179.889999  176.039993  179.270004   
56 2025-07-31 00:00:00-04:00  182.899994  183.300003  175.929993  177.869995   
57 2025-08-01 00:00:00-04:00  174.089996  176.539993  170.889999  173.720001   
58 2025-08-04 00:00:00-04:00  175.160004  180.199997  174.520004  180.000000   
59 2025-08-05 00:00:00-04:00  179.619995  180.259995  175.899994  178.259995   

       Volume  Dividends  Stock Splits     RSI_14      MACD  MACD_Signal  \
55  174312200        0.0           0.0  73.874718  7.097559     7.076429   
56  221685400        0.0           0.0  70.015438  7.019476     7.065038   
57  204529000        0.0           0.0  63.534344  6.547252     6.961481   
58  148174600        0.0           0.0  63.172796  6.603633     6.889911   
59  152248661        0.0           0.0  59.472081  6.433746     6.798678   

    MACD_Hist  
55   0.021130  
56  -0.045562  
57  -0.414229 